## import package

In [ ]:
!pip install torch
!pip install pandas nltk
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.6 MB/s 
     |████████████████████████████████| 182 kB 50.1 MB/s 
     |████████████████████████████████| 7.6 MB 64.9 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
import math
import numpy as np
import time
import torch, pandas as pd
import nltk
import re
nltk.download('punkt')

from transformers import set_seed
set_seed(777)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


cuda:0


In [ ]:
# Training data file
directory="/content/drive/MyDrive/AICUP/"


file=directory+"Batch_answers - train_data (no-blank).csv"
df=pd.read_csv(file, encoding = "ISO-8859-1")

In [ ]:
df = df.drop(columns=['Unnamed: 6', 'total no.: 7987'])

In [ ]:
df.head()

,id,q,r,s,q',r'
0,8,"""It can go both ways . We all doubt . It is wh...","""True .""",AGREE,"""It can go both ways . We all doubt . It is wh...","""True ."""
1,8,"""It can go both ways . We all doubt . It is wh...","""True .""",AGREE,"""can go both ways . We all doubt . It is what ...","""True"""
2,8,"""It can go both ways . We all doubt . It is wh...","""True .""",AGREE,"""It can go both ways . We all doubt . It is wh...","""True"""
3,9,"""once again , you seem to support the killing ...","""based on the idea that people are dispensible...",AGREE,"""seem to support the killing of certain people""","""based on the idea that people are dispensible..."
4,9,"""once again , you seem to support the killing ...","""based on the idea that people are dispensible...",AGREE,"""you seem to support the killing of certain pe...","""based on the idea that people are dispensible"""


## Data process

In [ ]:
df[['q','r',"q'","r'"]] = df[['q','r',"q'","r'"]].apply(lambda x: x.str.strip('\"'))

In [ ]:
df['r'] = df['s'] + ':' + df['r']

In [ ]:
df['sub_q_true'] = [1 if x in y else 0 for x,y in zip(df["q'"],df["q"])]
df['sub_r_true'] = [1 if x in y else 0 for x,y in zip(df["r'"],df["r"])]
df['sub_both'] = df['sub_q_true']*df['sub_r_true']

In [ ]:
df.head()

,id,q,r,s,q',r',sub_q_true,sub_r_true,sub_both
0,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,It can go both ways . We all doubt . It is wha...,True .,1,1,1
1,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,can go both ways . We all doubt . It is what y...,True,1,1,1
2,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,It can go both ways . We all doubt . It is wha...,True,1,1,1
3,9,"once again , you seem to support the killing o...",AGREE:based on the idea that people are dispen...,AGREE,seem to support the killing of certain people,based on the idea that people are dispensible ...,1,1,1
4,9,"once again , you seem to support the killing o...",AGREE:based on the idea that people are dispen...,AGREE,you seem to support the killing of certain peo...,based on the idea that people are dispensible,1,1,1


In [ ]:
data = df.loc[df['sub_both'] == 1]

In [ ]:
data['q_start'] = [y.index(x) for x,y in zip(data["q'"],data["q"])]
data['r_start'] = [y.index(x) for x,y in zip(data["r'"],data["r"])]
data['q_end'] = [x+len(y)-1 for x,y in zip(data["q_start"],data["q'"])]
data['r_end'] = [x+len(y)-1 for x,y in zip(data["r_start"],data["r'"])]

<ipython-input-12-2e5ebdb9e59b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['q_start'] = [y.index(x) for x,y in zip(data["q'"],data["q"])]
<ipython-input-12-2e5ebdb9e59b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['r_start'] = [y.index(x) for x,y in zip(data["r'"],data["r"])]
<ipython-input-12-2e5ebdb9e59b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
data.head()

,id,q,r,s,q',r',sub_q_true,sub_r_true,sub_both,q_start,r_start,q_end,r_end
0,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,It can go both ways . We all doubt . It is wha...,True .,1,1,1,0,6,76,11
1,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,can go both ways . We all doubt . It is what y...,True,1,1,1,3,6,74,9
2,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,It can go both ways . We all doubt . It is wha...,True,1,1,1,0,6,76,9
3,9,"once again , you seem to support the killing o...",AGREE:based on the idea that people are dispen...,AGREE,seem to support the killing of certain people,based on the idea that people are dispensible ...,1,1,1,17,6,61,98
4,9,"once again , you seem to support the killing o...",AGREE:based on the idea that people are dispen...,AGREE,you seem to support the killing of certain peo...,based on the idea that people are dispensible,1,1,1,13,6,81,50


In [ ]:
train = data

## Tokenizer

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('nbroad/ESG-BERT')

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
train_encodings = tokenizer(train['q'].tolist(), train['r'].tolist(), truncation=True, padding=True)

In [ ]:
tokenizer.decode(train_encodings['input_ids'][0])

'[CLS] it can go both ways. we all doubt. it is what you do with it that matters. [SEP] agree : true. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [ ]:
train_answer = train[['q_start', 'r_start',	'q_end', 'r_end']].to_dict('records')

In [ ]:
def add_token_positions(encodings, answers):
    q_start, r_start, q_end, r_end = [],[],[],[]

    for i in range(len(answers)):
        q_start.append(encodings.char_to_token(i, answers[i]['q_start'], 0))
        r_start.append(encodings.char_to_token(i, answers[i]['r_start'], 1))
        q_end.append(encodings.char_to_token(i, answers[i]['q_end'], 0))
        r_end.append(encodings.char_to_token(i, answers[i]['r_end'], 1))

        if q_start[-1] is None:
            q_start[-1] = 0
            q_end[-1] = 0
            # continue

        if r_start[-1] is None:
            r_start[-1] = 0
            r_end[-1] = 0
            # continue

        shift = 1
        while q_end[-1] is None:
            q_end[-1] = encodings.char_to_token(i, answers[i]['q_end'] - shift)
            shift += 1
        shift = 1
        while r_end[-1] is None:
            r_end[-1] = encodings.char_to_token(i, answers[i]['r_end'] - shift)
            shift += 1
    encodings.update({'q_start':q_start, 'r_start':r_start,	'q_end':q_end, 'r_end':r_end})

In [ ]:
# Convert char_based_id to token_based_id
# Find the corossponding token id after input being tokenized
add_token_positions(train_encodings, train_answer)

In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
print(train_encodings['q_start'][0])
print(train_encodings['r_start'][0])
print(train_encodings['q_end'][0])
print(train_encodings['r_end'][0])

1
26
20
27


## Dataset

In [ ]:


class qrDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
train_dataset = qrDataset(train_encodings)

In [ ]:
next(iter(train_dataset))

{'input_ids': tensor([    0,   243,    64,   213,   258,  1319,   479,   166,    70,  2980,
           479,    85,    16,    99,    47,   109,    19,    24,    14,  3510,
           479,     2,     2,  3450, 30009,    35, 36948,   479,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

## Model

In [ ]:
from transformers import AutoModelForSequenceClassification
class myModel(torch.nn.Module):

    def __init__(self):

        super(myModel, self).__init__()

        self.bert = AutoModelForSequenceClassification.from_pretrained('nbroad/ESG-BERT')
        # self.fc = nn.Linear(768, 26)
        

    def forward(self, input_ids, attention_mask):

        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        logits = output[0]
        out = self.fc(logits)

        return out



## Training

In [ ]:
from transformers import AdamW
from tqdm import tqdm

# Set GPU / CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# Put model on device
model = myModel().to(device)

optim = AdamW(model.parameters(), lr=1e-5)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
print(model)

In [ ]:
# Pack data into dataloader by batch
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
training_epoch = 3

In [ ]:
loss_fct = CrossEntropyLoss()

In [ ]:
for epoch in range(training_epoch):
    model.train()
    running_loss = 0.0

    loop = tqdm(train_loader, leave=True)
    for batch_id, batch in enumerate(loop):
        # reset
        optim.zero_grad()


        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        q_start = batch['q_start'].to(device)
        r_start = batch['r_start'].to(device)
        q_end = batch['q_end'].to(device)
        r_end = batch['r_end'].to(device)


        # model output
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        q_start_logits, r_start_logits, q_end_logits, r_end_logits = torch.split(outputs, 1, 2)

        q_start_logits = q_start_logits.squeeze(-1).contiguous()
        r_start_logits = r_start_logits.squeeze(-1).contiguous()
        q_end_logits = q_end_logits.squeeze(-1).contiguous()
        r_end_logits = r_end_logits.squeeze(-1).contiguous()

        q_start_loss = loss_fct(q_start_logits, q_start)
        r_start_loss = loss_fct(r_start_logits, r_start)
        q_end_loss = loss_fct(q_end_logits, q_end)
        r_end_loss = loss_fct(r_end_logits, r_end)



        loss = q_start_loss + r_start_loss + q_end_loss + r_end_loss

        # calculate loss
        loss.backward()
        # update parameters
        optim.step()

        running_loss += loss.item()
        if batch_id % 50 == 0 and batch_id != 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(
                batch_id + 1, batch_id, running_loss / 50))
            running_loss = 0.0

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

Epoch 0:   3%|▎         | 51/1862 [01:19<47:20,  1.57s/it, loss=10.4]

Epoch 51 Batch 50 Loss 18.9878


Epoch 0:   5%|▌         | 101/1862 [02:35<45:12,  1.54s/it, loss=10.1]

Epoch 101 Batch 100 Loss 9.3234


Epoch 0:   8%|▊         | 151/1862 [03:52<43:29,  1.53s/it, loss=11.3]

...

Epoch 2:  97%|█████████▋| 1801/1862 [46:00<01:33,  1.53s/it, loss=7.03]

Epoch 1801 Batch 1800 Loss 6.8622


Epoch 2:  99%|█████████▉| 1851/1862 [47:17<00:16,  1.54s/it, loss=6.92]

Epoch 1851 Batch 1850 Loss 7.3824


Epoch 2: 100%|██████████| 1862/1862 [47:33<00:00,  1.53s/it, loss=6.99]


In [ ]:
torch.save(model.state_dict(), directory + 'robertax2_5e-4_model')

In [ ]:
# model = myModel().to(device)
# model.load_state_dict(torch.load(directory + 'aicup_model'))

## test data process

In [ ]:
# Testing data file
directory="/content/drive/MyDrive/AICUP/"
file=directory+"Batch_answers - test_data(no_label).csv"
df=pd.read_csv(file, encoding = "ISO-8859-1")

In [ ]:
df[['q','r']] = df[['q','r']].apply(lambda x: x.str.strip('\"'))

In [ ]:
df.head()

,id,q,r,s
0,6199,-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -...,"If so , why do we still have apes , and why ar...",DISAGREE
1,5807,There 's a lot of discussion there on that iss...,Of course . The makers of Expelled were within...,DISAGREE
2,8487,`` It 's not helping . The guns these people h...,"Oh , I would wager about like Mexico , about 8...",DISAGREE
3,1760,Shooting : 3 seriously injured in Auburn shoot...,Pickup strikes group of four youths | Houston ...,AGREE
4,6228,This is the argument concerning 'choice ' that...,I believe there is a point at which we ( socie...,DISAGREE


In [ ]:
df['r'] = df['s'] + ':' + df['r']

In [ ]:
df.head()

,id,q,r,s
0,6199,-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -...,"DISAGREE:If so , why do we still have apes , a...",DISAGREE
1,5807,There 's a lot of discussion there on that iss...,DISAGREE:Of course . The makers of Expelled we...,DISAGREE
2,8487,`` It 's not helping . The guns these people h...,"DISAGREE:Oh , I would wager about like Mexico ...",DISAGREE
3,1760,Shooting : 3 seriously injured in Auburn shoot...,AGREE:Pickup strikes group of four youths | Ho...,AGREE
4,6228,This is the argument concerning 'choice ' that...,DISAGREE:I believe there is a point at which w...,DISAGREE


In [ ]:
test = df

## test tokenized

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
test_data_q = test['q'].tolist()
test_data_r = test['r'].tolist()

In [ ]:
test_encodings = tokenizer(test_data_q, test_data_r, truncation=True, padding=True)

In [ ]:
tokenizer.decode(test_encodings['input_ids'][0])

'<s>-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- ( Time. )</s></s>DISAGREE:If so, why do we still have apes, and why are there no transitional species that we can currently observe. Statistical averages say we should see a few hundred within the last two centuries, at a minimum.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

## test dataset & dataloader

In [ ]:
class qrDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
test_dataset = qrDataset(test_encodings)

In [ ]:
batch_size = 8
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## test predict

In [ ]:
model = myModel().to(device)
model.load_state_dict(torch.load(directory + 'robertax2_5e-4_model'))

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [ ]:
from transformers import AdamW
from tqdm import tqdm
def predict(test_loader):
    predict_pos = []

    model.eval()

    q_sub_output, r_sub_output = [],[]

    loop = tqdm(test_loader, leave=True)
    for batch_id, batch in enumerate(loop):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        # model output
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        q_start_logits, r_start_logits, q_end_logits, r_end_logits = torch.split(outputs, 1, 2)

        q_start_logits = q_start_logits.squeeze(-1).contiguous()
        r_start_logits = r_start_logits.squeeze(-1).contiguous()
        q_end_logits = q_end_logits.squeeze(-1).contiguous()
        r_end_logits = r_end_logits.squeeze(-1).contiguous()

        q_start_prdict = torch.argmax(q_start_logits, 1).cpu().numpy()
        r_start_prdict = torch.argmax(r_start_logits, 1).cpu().numpy()
        q_end_prdict = torch.argmax(q_end_logits, 1).cpu().numpy()
        r_end_prdict = torch.argmax(r_end_logits, 1).cpu().numpy()

        for i in range(len(input_ids)):
            predict_pos.append((q_start_prdict[i].item(), r_start_prdict[i].item(), q_end_prdict[i].item(), r_end_prdict[i].item()))

            q_sub = tokenizer.decode(input_ids[i][q_start_prdict[i]:q_end_prdict[i]+1])
            r_sub = tokenizer.decode(input_ids[i][r_start_prdict[i]:r_end_prdict[i]+1])
            
            q_sub_output.append(q_sub)
            r_sub_output.append(r_sub)
    
    return q_sub_output, r_sub_output, predict_pos



In [ ]:
q_sub_output, r_sub_output, predict_pos = predict(test_loader)

100%|██████████| 252/252 [01:02<00:00,  4.05it/s]


In [ ]:
def get_output_post_fn(test, q_sub_output, r_sub_output):
    q_sub, r_sub = [], []
    for i in range(len(test)):

        q_sub_pred = q_sub_output[i].split()
        r_sub_pred = r_sub_output[i].split()

        if q_sub_pred is None:
            q_sub_pred = []
        q_sub_error_index = q_sub_pred.index('[SEP]') if '[SEP]' in q_sub_pred else -1

        if q_sub_error_index != -1:
            q_sub_pred = q_sub_pred[:q_sub_error_index]

        temp = r_sub_pred.copy()
        if r_sub_pred is None:
            r_sub_pred = []
        else:
            for j in range(len(temp)):
                if temp[j] == '[SEP]':
                    r_sub_pred.remove('[SEP]')
                if temp[j] == '[PAD]':
                    r_sub_pred.remove('[PAD]')

        q_sub.append(' '.join(q_sub_pred))
        r_sub.append(' '.join(r_sub_pred))

    return q_sub, r_sub

In [ ]:
q_sub, r_sub = get_output_post_fn(test, q_sub_output, r_sub_output)

In [ ]:
test['q_sub'] = q_sub
test['r_sub'] = r_sub

In [ ]:
test.head(50)

,id,q,r,s,q_sub,r_sub
0,6199,-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -...,"DISAGREE:If so , why do we still have apes , a...",DISAGREE,<s>,"If so, why do we still have apes, and why are ..."
1,5807,There 's a lot of discussion there on that iss...,DISAGREE:Of course . The makers of Expelled we...,DISAGREE,a private screening of a film is a private scr...,Of course. The makers of Expelled were within ...
2,8487,`` It 's not helping . The guns these people h...,"DISAGREE:Oh , I would wager about like Mexico ...",DISAGREE,"The guns these people have, they don't registe...","Oh, I would wager about like Mexico, about 80 ..."
3,1760,Shooting : 3 seriously injured in Auburn shoot...,AGREE:Pickup strikes group of four youths | Ho...,AGREE,You don't need no stinkin'motive! You got a gu...,Pickup strikes group of four youths | Houston ...
4,6228,This is the argument concerning 'choice ' that...,DISAGREE:I believe there is a point at which w...,DISAGREE,"This is the argument concerning 'choice'that, ...",I believe there is a point at which we ( socie...
5,3537,"My point is , people are prepared to admit tha...",DISAGREE:The bible is not a science textbook ....,DISAGREE,"My point is, people are prepared to admit that...",The bible is not a science textbook. I do not ...
6,3874,Originally Posted by wayneinFL Each owns his o...,"DISAGREE:According to your logic , an infant w...",DISAGREE,The fetus does not have a positive right to su...,"According to your logic, an infant would have ..."
7,6223,"Really , so you are totally knowable ? See abo...","DISAGREE:Oh irrational-boy , please do try to ...",DISAGREE,"Really, so you are totally knowable? See above...","Oh irrational-boy, please do try to understand..."
8,963,"In essence , it seems as though the central cl...","DISAGREE:I 'd say no to the first part , and y...",DISAGREE,it seems as though the central claim is that n...,"I 'd say no to the first part, and yes to the ..."
9,1420,"STUTTGART , Germany - A 17-year-old gunman dre...",AGREE:Well is n't that interesting . Germany h...,AGREE,A 17-year-old gunman dressed in black opened f...,Well isn't that interesting.


## csv output

In [ ]:
q_ans = test['q_sub']
r_ans = test['r_sub']
q_ans = ['\"' + q + '\"' for q in q_ans]
r_ans = ['\"' + r + '\"' for r in r_ans]

In [ ]:
print(q_ans)
print(r_ans)

['"<s>"', '"a private screening of a film is a private screening -- they can boot anyone out that they want."', '"The guns these people have, they don\'t register, they don\'t care, they\'re probably stolen, they\'re probably obtained illegally, in many cases they\'re prohibited, `` commented Chief Hanson. sinjin - What\'s with the probably ( s )? Have they attempted to trace handguns recovered from criminals? Wonder how many trace right back to the US?"', '"You don\'t need no stinkin\'motive! You got a gun? You got to keep it loaded! And, you got to shoot it -- at people! \'Cuz you got to love your gun and you got to shoot it to shows it that you loves it! And you got to shoot at people to prove your undying love!"', '"This is the argument concerning \'choice\'that, to me, rings untrue and is a potential weak point. The whole \'when does life begin/when is it human?\'isn\'t where those that favor woman\'s rights should stake their claim. I support a woman\'s right to control her body 

In [ ]:
ans = pd.DataFrame({"id": test['id'], "q": q_ans, "r": r_ans})
print(ans.head())

     id                                                  q  \
0  6199                                              "<s>"   
1  5807  "a private screening of a film is a private sc...   
2  8487  "The guns these people have, they don't regist...   
3  1760  "You don't need no stinkin'motive! You got a g...   
4  6228  "This is the argument concerning 'choice'that,...   

                                                   r  
0  "If so, why do we still have apes, and why are...  
1  "Of course. The makers of Expelled were within...  
2  "Oh, I would wager about like Mexico, about 80...  
3  "Pickup strikes group of four youths | Houston...  
4  "I believe there is a point at which we ( soci...  


In [ ]:
import os
os.chdir('/content/drive/My Drive/AICUP') #切換該目錄
os.listdir() #確認目錄內容

['「roberta_5e-4',
 'Batch_answers - train_data (no-blank).csv',
 'aicup_model',
 'aicup_model_takefirst',
 'Batch_answers - test_data(no_label).csv',
 'test-submit_ans.txt',
 'test-submit_ans.csv',
 'test-submit_copy.csv',
 'distiltbert_model',
 'test-submit_distiltbert.csv',
 'roberta_model',
 'test-submit_roberta.csv',
 'baseline3epoch_model',
 "don't_change_2022_AIcup_baseline",
 'test-submit_robertax3.csv',
 'test-submit_robertax2.csv',
 'test-submit_convbertx2.csv',
 'convbertx2',
 'convbertx2_model',
 'convbertx3_model',
 'test-submit_convbertx3.csv',
 'test-submit_baseline3epoch.csv',
 'convbertx3',
 'distiltbert 的副本',
 'electrax2',
 'longformer',
 'roberta 的副本',
 'robertax2_3.5e-5_model',
 'roberta_3.5e-5',
 'robertax2_1e-5_model',
 'robertax2_5e-4_model',
 'robertax2_model',
 '「roberta_1e-5']

In [ ]:
ans.to_csv('test-submit_robertax2_5e-4.csv', sep=',', index=False, encoding='utf-8')

## Grading

In [ ]:
def nltk_token_string(sentence):
    # print(sentence)
    tokens = nltk.word_tokenize(sentence)
    for i in range(len(tokens)):
        if len(tokens[i]) == 1:
            tokens[i] = re.sub(r"[!\"#$%&\'()*\+, -.\/:;<=>?@\[\\\]^_`{|}~]", '', tokens[i])
    while '' in tokens:
        tokens.remove('')
    tokens = ' '.join(tokens)
    return tokens

In [ ]:
def lcs(X, Y):
    X_, Y_ = [], []
    
    X_ = nltk_token_string(X)
    Y_ = nltk_token_string(Y)

    m = len(X_)
    n = len(Y_)
 
    # declaring the array for storing the dp values
    L = [[None]*(n + 1) for i in range(m + 1)]
 
    """Following steps build L[m + 1][n + 1] in bottom up fashion
    Note: L[i][j] contains length of LCS of X[0..i-1]
    and Y[0..j-1]"""
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0 :
                L[i][j] = 0
            elif X_[i-1] == Y_[j-1]:
                L[i][j] = L[i-1][j-1]+1
            else:
                L[i][j] = max(L[i-1][j], L[i][j-1])
 
    # L[m][n] contains the length of LCS of X[0..n-1] & Y[0..m-1]
    return L[m][n]


def acc(full, sub):
    common = lcs(full, sub)
    union = len(full) + len(sub) - common
    accuracy = float(common/union)

    return accuracy

In [ ]:
q_acc_sum = 0
r_acc_sum = 0
for i in range(test.shape[0]):
    q_accuracy = acc(test.iloc[i]["q'"], test.iloc[i]['q_sub'])
    r_accuracy = acc(test.iloc[i]["r'"], test.iloc[i]['r_sub'])

    q_acc_sum += q_accuracy
    r_acc_sum += r_accuracy

print("q accuracy: ", q_acc_sum/test.shape[0])
print("r accuracy: ", r_acc_sum/test.shape[0])